In [2]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math # 수학적으로 계산할 게 있어서 쓰는 모듈

# 셀레늄 자동화를 하겠다
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라


# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

# for문 진행 상황 체크 하겠다
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expresseion): 필요한 문자(알파벳,한글), 숫자, 특수기호 를 정제하여 추출하겠다 
import re

from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [58]:
# 영화배우 이름 검색
name = '이정재'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")

# 네이버 검색어 입력 후 검색
element = driver.find_element_by_name("query")
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)


# 필모그래피 클릭
driver.find_element_by_link_text("필모그래피").click()
time.sleep(1)

# 인기순 클릭
driver.find_element_by_link_text("인기순").click()
time.sleep(1)

film_num = int(driver.find_element_by_css_selector('.this_text_number').text)
print('필모 수: ', film_num)

# 페이지 클릭횟수 계산
page_click = math.ceil(film_num/4) - 1
print('페이지 클릭 횟수: ', page_click)


# 영화 데이터 수집
title_list = []
score_list = []
cast_list = []
audiance_list = []
main_sub_list = []

for i in range(page_click):    
    # 제목
    titles = driver.find_elements_by_css_selector('.this_text')
    temp_list = []
    for title in titles:
        temp_list.append(title.text)
        temp_list = [x for x in temp_list if x !='' and x != '상영중']
#     print(temp_list)    
    title_list = title_list + temp_list    
    
    # 배역
    cast_temp = []
    for t in range(1, 5):
        try:
            cast = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > span'.format(i+1, t)).text[:2]
            cast_temp.append(cast)
        except:
            break
    cast_list = cast_list + cast_temp        
#     print(cast_temp)
    

    # 관객수
    score_temp = []
    for j in range(1, 5):
        try:
            score = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > div.info > dl:nth-child(2)'.format(i+1, j)).text.split('\n')[1]
            score = score.replace('만','0000').replace(',','')
            if '.' in score:
                score = int(score.replace('.', ''))
                score = round(score*0.1)
            else:
                score = round(int(score))
            
            score_temp.append(score)
        except:
            break
    score_list = score_list + score_temp
#     print(score_temp, '\n')
    if len(score_temp) < 4:
        break
        
    # 필모 다음페이지 클릭
    driver.find_element_by_css_selector('.pg_next.on').click()
    time.sleep(2)

title_list = title_list[:len(score_list)]
cast_list = cast_list[:len(score_list)]

# print('\n', len(title_list))
# print(title_list)
# print(cast_list)
# print(score_list)

# 
df = pd.DataFrame(list(zip(title_list, cast_list, score_list)), columns = ['제목','배역', 'score'])
print('관객수 데이터가 있는 영화들만: ')
print(df)

# 주연 스코어 합
cast_main = df[(df['배역']=='주연')]
main_score = cast_main['score'].sum()

# 조연 스코어 합
cast_sub = df[(df['배역']=='조연')]
sub_score = cast_sub['score'].sum()

# 단역 스코어 합
cast_other = df[(df['배역']!='주연') & (df['배역']!='조연')]
cast_score = cast_other['score'].sum()

# 배우 흥행지수
actor_score_index = round((main_score*0.5 + sub_score*0.4 + cast_score*0.1) / len(df))
print('{} 배우 흥행지수: '.format(name), actor_score_index)

필모 수:  36
페이지 클릭 횟수:  8
관객수 데이터가 있는 영화들만: 
                         제목  배역     score
0                 신과함께-죄와 벌  특별  14410000
1                       도둑들  주연  12980000
2                        암살  주연  12700000
3                 신과함께-인과 연  특별  12270000
4                        관상  주연   9130000
5                    인천상륙작전  주연   7050000
6                       신세계  주연   4680000
7               다만 악에서 구하소서  주연   4350000
8                        태풍  주연   3470000
9                       사바하  주연   2390000
10                       하녀  주연   2260000
11                      빅매치  주연   1170000
12                      대립군  주연    830000
13              1724 기방난동사건  주연    280000
14                  어쩌다, 결혼  특별     77000
15         다만 악에서 구하소서 파이널컷  주연     38000
16  인천상륙작전 : 익스텐디드 에디션(확장판)  주연     27000
이정재 배우 흥행지수:  1961953
